# Load the stored data

In this section, we will load the stored pickled data into memory.

First, set the names of the pickled files to load:

In [ ]:
cell_names_file_name = "../data/cell_names"
gene_names_file_name = "../data/gene_names"
labels_file_name = "../data/labels"
gene_expr_bin_bitpacked_file_name = "../data/gene_expr_bin_bitpacked"

Then, actually load the data in those files

In [ ]:
from ctc_utils.functions import load_pickled_cells_data

(cell_names, gene_names, labels, gene_expr_bin) = load_pickled_cells_data(
    cell_names_file_name,
    gene_names_file_name,
    labels_file_name,
    gene_expr_bin_bitpacked_file_name + ".npy",
)


# Partition into seperate datasets

Define gene expressions per dataset

In [ ]:

ge_10xv2 = gene_expr_bin[0:6443]
ge_SM2 = gene_expr_bin[6444:6696]
ge_10xv3 = gene_expr_bin[6697:9918]
ge_CL = gene_expr_bin[9919:10171]
ge_DR = gene_expr_bin[10172:13393]
ge_iD = gene_expr_bin[13394:16615]
ge_SW = gene_expr_bin[16616:19791]
ge_10xv2_2 = gene_expr_bin[19792:23153]

Define labels per dataset

In [ ]:
lb_10xv2 = labels[0:6443]
lb_SM2 = labels[6444:6696]
lb_10xv3 = labels[6697:9918]
lb_CL = labels[9919:10171]
lb_DR = labels[10172:13393]
lb_iD = labels[13394:16615]
lb_SW = labels[16616:19791]
lb_10xv2_2 = labels[19792:23153]

# Choose datasets

In [ ]:
import numpy as np

# Define dataset to be used
x = ge_SW
y = lb_SW
# Only useful if test set will be different than training set.
x_dt_test = ge_iD
y_dt_test = lb_iD

# Balance datasets (Step 1)

Find amount of occurances for the majority class

(Comment out if you dont wanna use it)

In [ ]:
from collections import  Counter
import numpy as np
# d = Counter(y)
# print(d)
# max_occurances = max(d, key=d. get)
# max_num = d.get(max_occurances)
# half = int(max_num/2)


Prune all classes so that the maximum amount of their occurances matches the required threshold (ahlf the amount of occurances of the majority class).
You can specify your own!

In [ ]:
from collections import  Counter
import numpy as np
from ctc_utils.functions import prune_training_set

# x, y = prune_training_set(x, y, half)




# Feature selection

In [ ]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=0.16)
x = sel.fit_transform(x)
x_dt_test = sel.transform(x_dt_test)


# Data Overview

Plot cell type number of occurances in each dataset.

In [ ]:
from ctc_utils.functions import plot_cell_type_counts
from collections import Counter

Dataset_Names = ["10xv2","SM2","10xv3","CL","DR","iD","SW","10xv2_2"]
Dataset_Labels = [lb_10xv2, lb_SM2, lb_10xv3, lb_CL,lb_DR, lb_iD,lb_SW,lb_10xv2_2]
print(gene_expr_bin.shape)
for indx in range(len(Dataset_Names)):
    print("Dataset: " + Dataset_Names[indx], len(Dataset_Labels[indx]) )
    print(Counter(Dataset_Labels[indx]))
    plot_cell_type_counts(Dataset_Labels[indx])
    print("\n")

print("Training Dataset")
plot_cell_type_counts(y)

print("Test Dataset")
plot_cell_type_counts(y_dt_test)

# Divide the test and training data

From the loaded data, divide it into training and test data

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42
)

# (Optional) Test with different datasets

Comment out, if you want to evaluate performance only using the reference set (partitioning).

In [ ]:
x_train = x
y_train = y
x_test = x_dt_test
y_test = y_dt_test

# Apply oversampling (Balance dataset step 2)

!! Can be used also to Balance the reference set (Balance dataset step 2) !!

(if x_over and y_over are not used when fitting the model, oversampling is not used).

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN,  SMOTEN
from ctc_utils.functions import plot_cell_type_counts 

over_sampling = RandomOverSampler(sampling_strategy = "not majority")
 # fit and apply the transform
x_over, y_over = over_sampling.fit_resample(x_train, y_train)

plot_cell_type_counts(y_over)


# Set model name
(Example)
Define the name of the model to be loaded or to be saved.
(In case you want to save any of the models)

In [ ]:
model_file_name_knn = "../models/knn"
model_file_name_linreg = "../models/linreg"


# Apply KNN

In [ ]:
def dinstanceMetric(a,b):
    # to use add this:  metric='pyfunc', metric_params = {"func" :dinstanceMetric}
    return np.var(np.subtract(a,b))/(np.var(a)+ np.var(b))

from sklearn import neighbors
from sklearn import neighbors
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics  import f1_score

n = 16

steps = [('tsvd', TruncatedSVD(n_components = 11)), ('knn',neighbors.KNeighborsClassifier(n,n_jobs=-1))]
model = Pipeline(steps=steps, verbose=1)

#apply knn
model.fit(x_train,y_train)

y_pred = model.predict(x_test) 
#Print f1
print(f1_score(y_pred, y_test, average= "weighted"))

from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

plot_confusion_matrix(model, x_test, y_test, xticks_rotation = "vertical", normalize = "true", values_format = ".2f")  
plt.show()

import pickle

# Save the model example
pickle.dump(model, open(model_file_name_knn, "wb"))

# Apply Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import FactorAnalysis
from sklearn.metrics  import f1_score




steps = [('tsvd', TruncatedSVD(n_components=55)), ('m', LogisticRegression(n_jobs=-1))]
model = Pipeline(steps=steps)
model.fit(x_train,y_train)

#plot confusion matrix
plot_confusion_matrix(model, x_test, y_test, xticks_rotation = "vertical", normalize = "true", values_format = ".2f")  
plt.show()


#Print f1
print(f1_score(model.predict(x_test), y_test, average= "weighted"))

import pickle

# Save the model example
pickle.dump(model, open(model_file_name_linreg, "wb"))

# Apply Decision Tree Class classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

steps = [('svd', FactorAnalysis(n_components=55)), ('m', DecisionTreeClassifier(criterion = "entropy"))]
model = Pipeline(steps=steps)

model.fit(x_train,y_train)

# Print f1
print(f1_score(model.predict(x_test), y_test, average = "weighted"))

plot_confusion_matrix(model, x_test, y_test, xticks_rotation = "vertical", normalize = "true", values_format = ".2f")  
plt.show()


# Apply Linear SVC

In [ ]:
from sklearn import svm
from sklearn.decomposition import PCA

steps = [('fa', FactorAnalysis(n_components=65)), ('m', svm.LinearSVC())]
model = Pipeline(steps=steps)

model.fit(x_train,y_train)

# Print accuracy
print(f1_score(model.predict(x_test), y_test, average= "weighted"))

plot_confusion_matrix(model, x_test, y_test, xticks_rotation = "vertical", normalize = "true", values_format = ".2f")  
plt.show()


# Apply  Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

steps = [('tsvd', TruncatedSVD(n_components=60)), ('m', GaussianNB())]
model = Pipeline(steps=steps)

model.fit(x_train,y_train)

# Print accuracy
print(f1_score(model.predict(x_test), y_test, average= "weighted"))

plot_confusion_matrix(model, x_test, y_test, xticks_rotation = "vertical", normalize = "true", values_format = ".2f")  
plt.show()

# Apply MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

steps = [('svd',FactorAnalysis(n_components=65)), ('m', MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1))]
model = Pipeline(steps=steps)

model.fit(x_train,y_train)

# Print f1
print(f1_score(model.predict(x_test), y_test, average= "weighted"))

plot_confusion_matrix(model, x_test, y_test, xticks_rotation = "vertical", normalize = "true", values_format = ".2f")  
plt.show()

# SDG 

In [ ]:
from sklearn.linear_model import SGDClassifier

steps = [('svd',TruncatedSVD(n_components=150)), ('m', SGDClassifier(fit_intercept = False, class_weight = "balanced", average = True ))]
model = Pipeline(steps=steps)

model.fit(x_train,y_train)

# Print f1
print(f1_score(model.predict(x_test), y_test, average="weighted"))

plot_confusion_matrix(model, x_test, y_test, xticks_rotation = "vertical", normalize = "true", values_format = ".2f")  
plt.show()


# Stacking Classifiers

Define the classifiers to be used as part of the ensemble classifier

In [ ]:
logreg = LogisticRegression(n_jobs=-1)
dt = DecisionTreeClassifier(criterion = "entropy")
linearsvc = svm.LinearSVC()
sdg = SGDClassifier(fit_intercept = False, class_weight = "balanced", average = True )
knn = neighbors.KNeighborsClassifier(n,n_jobs=-1)


Run classification

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimator_list = [
    ('dt',dt),
    ('linearsvc',linearsvc),
    ('sdg',sdg)
   ]

stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=LogisticRegression()
)

stack_model.fit(x_train, y_train)

y_test_pred = stack_model.predict(x_test)


stack_model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score


Plot confusion matrix

In [ ]:
print(stack_model_test_f1)

plot_confusion_matrix(stack_model, x_test, y_test, xticks_rotation = "vertical", normalize = "true", values_format = ".2f")  
plt.show()